<a href="https://colab.research.google.com/github/flosrv/5-Days-Live-EDA-and-Feature-Engineering/blob/main/Python_Project_For_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import requests
import pandas as pd
import sqlite3
import datetime as dt
from bs4 import BeautifulSoup
from io import StringIO

In [91]:
# Constants
URL = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
EXCHANGE_RATE_URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
OUTPUT_CSV_PATH = './Largest_banks_data.csv'
DB_NAME = 'Banks.db'
TABLE_NAME = 'Largest_banks'

In [92]:
# Task 1: Logging function
LOG_FILE = 'code_log.txt'
def log_progress(message):
    timestamp = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f"{timestamp} - {message}"
    with open(LOG_FILE, 'a') as log_file:
        log_file.write(log_message + '\n')
    print(log_message)

In [93]:
# Task 2: Extract exchange rates CSV and create SQLite table
def extract_exchange_rates():
    log_progress("Starting exchange rate extraction...")
    exchange_rate_df = pd.read_csv(EXCHANGE_RATE_URL)
    conn = sqlite3.connect(DB_NAME)
    try:
        exchange_rate_df.to_sql('Exchange_Rates', conn, if_exists='fail', index=False)
    except ValueError:
        log_progress("Exchange Rates table already exists, skipping creation.")
    conn.close()
    log_progress("Exchange rates loaded into database.")

In [94]:
# Task 3: Extraction of data
def extract():
    log_progress("Starting extraction process...")
    response = requests.get(URL, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    html = str(table)
    print(html)
    table_data = StringIO(html)
    df = pd.read_html(table_data)[0]
    df.columns = df.columns.str.strip()
    df = df[['Bank name', 'Market cap (US$ billion)']]
    df.columns = ['Name', 'MC_USD_Billion']
    log_progress("Extraction complete.")
    return df

In [95]:
# Task 4: Transformation of data
def transform(df):
    log_progress("Starting transformation process...")
    conn = sqlite3.connect(DB_NAME)
    rates_df = pd.read_sql_query("SELECT * FROM Exchange_Rates", conn)
    conn.close()

    # Extract exchange rates from the database
    rates = rates_df.set_index('Currency')['Rate'].to_dict()

    # Calculate market caps in different currencies
    df['MC_GBP_Billion'] = (df['MC_USD_Billion'] * rates['GBP']).round(2)
    df['MC_EUR_Billion'] = (df['MC_USD_Billion'] * rates['EUR']).round(2)
    df['MC_INR_Billion'] = (df['MC_USD_Billion'] * rates['INR']).round(2)

    log_progress("Transformation complete.")
    return df

In [96]:
# Task 5: Loading to CSV
def load_to_csv(df):
    log_progress("Starting CSV load process...")
    df.to_csv(OUTPUT_CSV_PATH, index=False)
    log_progress(f"CSV file saved at {OUTPUT_CSV_PATH}.")

# Task 6: Loading to Database
def load_to_db(df):
    log_progress("Starting database load process...")
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            Name TEXT,
            MC_USD_Billion REAL,
            MC_GBP_Billion REAL,
            MC_EUR_Billion REAL,
            MC_INR_Billion REAL
        )
    ''')
    df.to_sql(TABLE_NAME, conn, if_exists='replace', index=False)
    log_progress(f"Data loaded into database table {TABLE_NAME}.")
    conn.commit()
    conn.close()

In [97]:
# Task 7: Function to run queries on the database
def run_queries():
    log_progress("Starting query execution...")
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    query = f"SELECT AVG(MC_GBP_Billion) FROM {TABLE_NAME} ;"
    print(f"Executing query: {query}")
    cursor.execute(query)
    results = cursor.fetchall()
    print(results)
    log_progress(results)
    log_progress("Query execution complete.")
    conn.close()

In [98]:
# Task 8: Verify log entries
def verify_log_entries():
    log_progress("Verifying log entries...")
    try:
        with open(LOG_FILE, 'r') as log_file:
            log_contents = log_file.read()
            if "Starting extraction process..." not in log_contents or \
               "Starting transformation process..." not in log_contents or \
               "Starting CSV load process..." not in log_contents or \
               "Starting database load process..." not in log_contents or \
               "Starting query execution..." not in log_contents:
                raise Exception("Log entries missing!")
            print("Log entries verified.")
            log_progress("Log entries verified.")
    except Exception as e:
        print(f"Error: {e}")
        log_progress(f"Error: {e}")

In [99]:
# Main execution flow
def main():
    extract_exchange_rates() # First, load exchange rates to the database
    df = extract()
    df = transform(df)
    load_to_csv(df)
    load_to_db(df)
    run_queries()
    verify_log_entries()

# Run the main function
if __name__ == '__main__':
    main()

2024-12-23 13:17:19 - Starting exchange rate extraction...
2024-12-23 13:17:20 - Exchange Rates table already exists, skipping creation.
2024-12-23 13:17:20 - Exchange rates loaded into database.
2024-12-23 13:17:20 - Starting extraction process...
<table class="wikitable sortable mw-collapsible">
<tbody><tr>
<th data-sort-type="number">Rank
</th>
<th>Bank name
</th>
<th>Market cap<br/>(US$ billion)
</th></tr>
<tr>
<td>1
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_o